
# Generate Readme with up to date list of xRunBooks

## Input

### Import librairies

In [ ]:
import os
import requests
import urllib.parse
import copy
import markdown
import nbformat
from nbconvert import MarkdownExporter
from papermill.iorw import (
    load_notebook_node,
    write_ipynb,
)
try:
    from git import Repo
except:
    !pip install GitPython
    from git import Repo

### Variables

In [ ]:
# README variables
readme_template = "README_template.md"
readme = "README.md"
replace_var = "[[DYNAMIC_LIST]]"

# welcome variables
#this is a TODO
#welcome_template = "Welcome_template.ipynb"
#welcome = "Welcome.ipynb"
#replace_var_quote = f'"[[DYNAMIC_LIST]]",\n'



# Others
current_file = '.'
notebook_ext = '.ipynb'
github_url = 'https://github.com/unskript/Awesome-CloudOps-Automation/tree/master'
local_OSS_url = 'http://127.0.0.1:8888/lab/tree'
#fix these!
github_download_url = 'https://raw.githubusercontent.com/unskript/Awesome-CloudOps-Automation/master/'
unSkript_logo ='https://unskript.com/assets/favicon.png'


### Get files list

In [ ]:
repo = Repo('.')
branch = repo.active_branch
list_of_dir = f"https://api.github.com/repos/unskript/Awesome-CloudOps-Automation/git/trees/{branch.name}?recursive=1"
r_gh = requests.get(list_of_dir).json().get("tree")
notebooks = []
actions=[]
for file in r_gh:
    if ".github" not in file.get("path") and ".gitignore" not in file.get("path") and "templates" not in file.get("path") and "/" in file.get("path")and "__init__" not in file.get("path"):
        if file.get("path").endswith(".ipynb") or file.get("path").endswith(".py"):
            temp = file.get("path").split("/")
            if temp == -1:
                data = {
                    "root": None,
                    "subdir": file.get("path")
                }
                notebooks.append(data)
                
            else:
                last_folder = ""
                file_name = temp[-1]
                temp.pop()
                for folder in temp:
                    last_folder += "/" + folder
                ##we now have a path.. but only really need the root folder
                ## different ways to generate fior action vs runbook
                slashcount  = last_folder.count("/")
                if slashcount == 1:
                    ##root folder for notebooks
                    root = last_folder[1:]
                else:
                    #this is an action folder
                    #find frist slash
                    firstslash = last_folder.find("/",1)
                    root = last_folder[1:firstslash]
                
                data = {
                    "root": root,
                    "subdir": file_name
                }
                if(file.get("path").endswith(".ipynb")):
                    notebooks.append(data)
                else:
                    actions.append(data)


## Model

### Reformat functions

In [ ]:
def reformat_file_name(file):
    file_nice = file.replace('_', ' ')
    file_nice = file_nice.replace('-', ' ')
    file_nice = file_nice.replace(notebook_ext, '')
    file_nice = file_nice.strip()
    if (file_nice != ""):
            file_nice = file_nice[0].capitalize() + file_nice[1:]
    return file_nice

### Get functions

In [ ]:

#title = get_title(folder_nice, file,file_nice, github_runbook_url, local_url)
def get_title(folder_nice, file,file_nice, github_runbook_url, local_url):
    return f'| {folder_nice} | [{file_nice}]({github_runbook_url}) | [Open in browser]({local_url})'

def get_list_item(folder_nice, file,file_nice, github_runbook_url, local_url):
    return f'* [{file_nice}]({github_runbook_url})'    

def create_directory_readme(folder_nice, runbook_list):
    # Open README template
    #template = open(directory_readme_template).read()
    
    #directory readme name
    directory_readme = f"{folder_nice}/README.md"
    print("directory_readme", directory_readme)
    template_header = f"<img align=\"center\" src=\"https://unskript.com/assets/favicon.png\" width=\"100\" height=\"100\" style=\"padding: 5px\">"    
    template = f"{template_header}\n\n # {folder_nice} xRunBooks\n\n{runbook_list}"
    print(template)
    
    # Save README
    f  = open(directory_readme, "w+")
    f.write(template)
    f.close()
    
def append_directory_readme(folder_nice, action_list):
    
    ##actions are written in a second loop - so append the readme - if it has the term xrunbook in it.
    #if there is no "xRunBook" - then this direcory only has actions and should overwrite the entire file
    
    # Open README (created with runbook list)
    #directory readme name
    directory_readme = f"{folder_nice}/README.md"
    template = open(directory_readme).read()
    print("directory_readme", directory_readme)
    
    if "xRunBook" in template:
        #we append the file
        ##now let's append in the Action list
        template = template + f"\n\n # {folder_nice} Actions \n\n{action_list}"
    else:
        #this file has no runbook data, so overwrite the file
        template =  f"\n\n # {folder_nice} Actions \n\n{action_list}"
    print(template)
    
    # Save README
    f  = open(directory_readme, "w+")
    f.write(template)
    f.close()

### Convert filepath in Markdown text

In [ ]:
def get_file_md(folder_nice, folder_url, files):
    good_format = 0
    final_title = ""
    md = ""
    folder_name = ""
    tool_name = ""
    tool_title = ""

    if (len(files) > 0):
       # md += f"\n{title_sep} {folder_nice}\n"
        for file in files:
            #for jupyter runbooks
            if file.endswith(notebook_ext):
                good_format = 0
                file_url = urllib.parse.quote(file)
                folder_name = folder_nice
                temp = folder_name.split("_")
                tool_name = temp[0]
                file_nice = reformat_file_name(file)
                
                #create the localhost url by combining the root with the file name
                local_url= f"{local_OSS_url}/{file_url}"
                github_runbook_url = f"{github_url}/{folder_name}/{file_url}"
                
                title = f"{get_title(folder_nice, file, file_nice, github_runbook_url, local_url)}\n"
                
                md += title
            #for actions - they end in .py
            if file.endswith(".py"):
                good_format = 0
                file_url = urllib.parse.quote(file)
                action_folder = file_url[:-3]
                folder_name = folder_nice
                temp = folder_name.split("_")
                tool_name = temp[0]
                file_nice = reformat_file_name(file)
                    
                #remove the "local url": there is no concept of local actions
            
                github_action_url = f"{github_url}/{folder_name}/legos/{action_folder}"
             
                title = f"* [{file_nice}]({github_action_url}) \n"
                
                md += title
    #print("md", md)
    return md

In [ ]:
def get_file_md_quote(folder_nice, folder_url, files):
    good_format = 0
    final_title = ""
    md = ""
    folder_name = ""
    tool_name = ""
    tool_title = ""

    if (len(files) > 0):
       # md += f"\n{title_sep} {folder_nice}\n"
        for file in files:
            if file.endswith(notebook_ext):
                good_format = 0
                file_url = urllib.parse.quote(file)
                folder_name = folder_nice
                temp = folder_name.split("_")
                tool_name = temp[0]
                file_nice = reformat_file_name(file)
                
                #create the localhost url by combining the root with the file name
                local_url= f"{local_OSS_url}/{file_url}"
                github_runbook_url = f"{github_url}/{folder_name}/{file_url}"        
                title = f'"{get_title(folder_nice, file, file_nice, github_runbook_url, local_url)}",\n'      
                md += title

   # print("md", md)
    return md

## get list of xRunbooks for subfolder Readme

In [ ]:
def get_file_list(folder_nice, folder_url, files):
    md = ""
    folder_name = ""
    tool_name = ""
    tool_title = ""

    if (len(files) > 0):
        for file in files:
            if file.endswith(notebook_ext):
                good_format = 0
                file_url = urllib.parse.quote(file)
                folder_name = folder_nice
                temp = folder_name.split("_")
                tool_name = temp[0]
                file_nice = reformat_file_name(file)
                
                #create the localhost url by combining the root with the file name
                local_url= f"{local_OSS_url}/{file_url}"
                github_runbook_url = f"{github_url}/{folder_name}/{file_url}"
                
                list_item = f"* [{file_nice}]({github_runbook_url})\n"
                md += list_item

   # print("md", md)
   # print(json_templates)
    return md

## get list of Actions for subfolder Readme

In [ ]:
def get_action_list(folder_nice, files):
    md = ""
    folder_name = ""
    tool_name = ""
    tool_title = ""

    if (len(files) > 0):
        for file in files:
            if file.endswith(".py"):
                good_format = 0
                file_url = urllib.parse.quote(file)
                folder_name = folder_nice
                file_nice = reformat_file_name(file)
                list_item = f"* {file_nice}\n"
                md += list_item

   # print("md", md)
   # print(json_templates)
    return md

### Generate markdown for each notebooks

In [ ]:
generated_list = ""
generated_list_quote = ""
generated_list_runbooks=""
json_templates = []
list_of_tools = []
index_max = len(notebooks)
index = 0

while index <= (index_max) - 1:
    folder_nice = notebooks[index].get("root")
    if folder_nice not in list_of_tools and folder_nice != "":
        #md_round is creating the table for the home readme
        md_round = ""
        md_round_quote=""
        #md_folder_readme is creating the simpler table for the subdirectory readme
        md_folder_readme = ''
        files = []
        list_of_tools.append(folder_nice)
        folder_url = urllib.parse.quote(folder_nice)
        print(folder_nice)
        while True:
            if notebooks[index].get("root") != folder_nice:
                break
            print(notebooks[index].get("subdir"))
            files.append(notebooks[index].get("subdir"))
            index += 1
            if index == index_max:
                break
            
        #create the table for the home readme
        md_round += get_file_md(folder_nice, folder_url, files )
        md_round_quote += get_file_md_quote(folder_nice, folder_url, files)
            
        #create a simple list for the subdirectory readme
        md_folder_readme += get_file_list(folder_nice, folder_url, files)
        #with this list - create the readme for the directory
        create_directory_readme(folder_nice, md_folder_readme)
    generated_list += md_round
    generated_list_runbooks += f"\n\n\n# {folder_nice}\n\n {md_folder_readme}"
    #print("generated_list",generated_list)
    generated_list_quote += md_round_quote

### Generate markdown for each Action

In [ ]:
generated_list_actions = ""
generated_list_quote_actions = ""
json_templates = []
list_of_tools = []
index_max = len(actions)
index = 0

while index <= (index_max) - 1:
    folder_nice = actions[index].get("root")
    if folder_nice not in list_of_tools and folder_nice != "":
        #md_round is creating the table for the home readme
        md_round = ""
        md_round_quote=""
        #md_folder_readme is creating the simpler table for the subdirectory readme
        md_folder_readme = ''
        files = []
        list_of_tools.append(folder_nice)
        folder_url = urllib.parse.quote(folder_nice)
       # print(folder_nice)
        while True:
            if actions[index].get("root") != folder_nice:
                break
      #      print(actions[index].get("subdir"))
            files.append(actions[index].get("subdir"))
            index += 1
            if index == index_max:
                break
        md_round += get_file_md(folder_nice, folder_url, files )    
        #print("stuff", folder_nice, files)
        md_folder_readme += get_action_list(folder_nice, files)
        print("md_folder_readme", md_folder_readme)
        #with this list - create the readme for the directory
        append_directory_readme(folder_nice, md_round)
        generated_list_actions += f"\n\n\n# {folder_nice}\n\n {md_round}"
#print("generated_list_actions\n",generated_list_actions)

## Output

### Preview the generated list

### Generate readme for github repository

In [ ]:
# Open README template
template = open(readme_template).read()

# Replace var to get list of templates in markdown format
template = template.replace(replace_var, generated_list)

# Save README
f  = open(readme, "w+")
f.write(template)
f.close()

In [ ]:

#generate a page of all the actions that are available
action_list = readme = "lists/Action_list.md"
# Save README
f  = open(action_list, "w+")
f.write(generated_list_actions)
f.close()


In [ ]:
#generate a page of all the Runbooks that are available
rb_list = readme = "lists/xRunBook_list.md"
# Save README
f  = open(rb_list, "w+")
f.write(generated_list_runbooks)
f.close()